## Step 1. Data 다운로드

- 아래 링크에서 **Song Lyrics** data download
- 저장된 파일 **압축 해제**한 후, 모든 ```txt``` 파일을 ```lyrics```폴더를 만들어 그 안에 저장

-  wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
- unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기

## Step 2. Data 읽어오기

- ```glob``` 모듈을 사용하면 파일 읽엉는 작업 용이
- ```glob``` 활용하여 모든 ```txt``` 파일 읽어온 후
- ```raw_corpus``` 리스트에 문장 단위로 저장

In [1]:
import re
import numpy as np
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))    # element 개수 = 데이터 크기 
print("Examples:\n", raw_corpus[:9])    # 앞에서부터 10라인 화면 출력

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.', 'It was too wet to play.', 'So we sat in the house', 'All that cold cold wet day.', 'I sat there with Sally.', 'We sat there we two.', 'And I said How I wish']


## Step 3. Data 정제

- ```preprocess_settence()```함수 활용해서 데이터 정제
- 지나치게 긴 문장은 다른 데이터들이 과도한 padding을 갖게 하므로 제거
- 문장을 **토큰화** 했을 때 토큰 개수가 15개 넘어가면 자르기 권유

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  
    
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("The @#$ SUN d d d d D d a243414#@$d d d d d  "))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> the sun d d d d d d a d d d d d <end>


## Step 4. 평가 데이터셋 분리
- ```tokenize()```함수로 데이터를 tensor로 변환 후
- ```sklearn```모듈의 ```train_test_split()```함수를 사용하여 훈련, 평가 데이터 분리
- **단어장의 크기는 12,000 이상**으로 설정
- **총 데이터의 20%**를 평가 데이터셋으로 사용

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=1200,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   6 904 ...   0   0   0]
 [  2 122   1 ...   0   0   0]
 [  2   6 305 ...   0   0   0]
 ...
 [  2 675  27 ...   0   0   0]
 [  2 675  27 ...   0   0   0]
 [  2 675  27 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f66a9252050>


In [5]:
tensor.shape

(175986, 347)

In [6]:
tensor = np.delete(tensor, np.s_[14:], axis=1)

In [7]:
tensor.shape

(175986, 14)

In [8]:
tensor

array([[  2,   6, 904, ...,   0,   0,   0],
       [  2, 122,   1, ...,   0,   0,   0],
       [  2,   6, 305, ...,   0,   0,   0],
       ...,
       [  2, 675,  27, ..., 675,  27,   6],
       [  2, 675,  27, ...,   0,   0,   0],
       [  2, 675,  27, ...,   0,   0,   0]], dtype=int32)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(tensor, test_size=0.2, random_state=34)

In [10]:
print(x_train.shape)
print(x_valid.shape)

(140788, 14)
(35198, 14)


In [11]:
enc_train = x_train[:, :-1] 
dec_train = x_train[:, 1:]

enc_val = x_valid[:, :-1]
dec_val = x_valid[:, 1:]

In [12]:
print(enc_train.shape)
print(dec_train.shape)

print(enc_val.shape)
print(dec_val.shape)

(140788, 13)
(140788, 13)
(35198, 13)
(35198, 13)


## Step 5. 인공지능 만들기
- 모델의 Embedding Size & Hidden Size 조절
- 10 Epoch 안에 ```val_loss```값을 **2.2**수준으로 줄이는 모델 설계
- Loss는 제시된 Loss 함수 그대로 사용!

In [13]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

In [14]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        self.bn2a_1 = tf.keras.layers.BatchNormalization()

        
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.bn2a_1(out)
        out = self.rnn_2(out)
        
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
for enc_sample, dec_sample in dataset.take(1): break
model(enc_sample)

<tf.Tensor: shape=(256, 13, 1201), dtype=float32, numpy=
array([[[-6.1928446e-04,  4.9484993e-04, -5.6126277e-04, ...,
         -7.2491291e-04, -2.2428794e-04,  2.2621317e-04],
        [-1.4983560e-04,  6.2973663e-04, -9.5036725e-04, ...,
         -1.4311346e-03, -4.8766451e-04,  8.3320704e-04],
        [ 5.9212546e-04,  5.1433942e-04, -1.1341163e-03, ...,
         -2.5185011e-03, -4.7088266e-05,  1.2627222e-03],
        ...,
        [ 1.6239014e-03, -1.6476064e-03, -3.9961352e-03, ...,
         -1.1682006e-03, -2.5319663e-04,  1.5639492e-04],
        [ 1.4218043e-03, -8.7633426e-04, -4.1384338e-03, ...,
         -8.8077725e-04, -1.3841821e-04, -7.7168789e-04],
        [ 1.9771541e-03, -6.6596369e-04, -4.7851796e-03, ...,
         -1.2539623e-04, -2.9328893e-04, -1.0832996e-03]],

       [[-6.1928446e-04,  4.9484993e-04, -5.6126277e-04, ...,
         -7.2491291e-04, -2.2428794e-04,  2.2621317e-04],
        [-4.4212784e-04,  1.7335743e-04, -1.0342548e-03, ...,
         -1.1343705e-03,  

- model의 최종 출력 tensor ```shape = (256, 13, 18001)```
- 18001: Dense layer의 출력 차원수
- 18001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링 해야하기 때문    
<br/>   
- 256: 배치 사이즈
- ```dataset.take(1)```를 통해 1개 배치, 256개의 문장에서 가져온 것    
<br/>    
- 13: ```tf.keras.lauyers.LSTM(hidden_size, return_sequences=True)```로 호출한 LSTM layer에서 ```return_sequences=True```이정 한 부분
- LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력
- ```return_sequences=False``` 였다면, LSTM layer는 1개의 벡터만 출력   
<br/>   
- 문제점: 우리 모델은 입력 데이터의 스퀀스 길이를 모름
- 데이터를 입력받으면서 13을 알게 됨
- 데이터 셋의 max_len이 13으로 맞춰져 있었던 것

In [17]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  307456    
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  1231025   
_________________________________________________________________
batch_normalization (BatchNo multiple                  4096      
Total params: 15,182,257
Trainable params: 15,180,209
Non-trainable params: 2,048
_________________________________________________________________


- ouput shape를 정확하게 알려주지 않음
- 우리 모델은 입력 시퀀스의 길이를 모르기 때문에 output shape를 특정할 수 없음
- 하지만, 모델의 파라미터 사이즈는 측정 가능
- 약 32million

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 44s 80ms/step - loss: 2.8287 - val_loss: 3.6007
Epoch 2/10
549/549 [==============================] - 45s 81ms/step - loss: 2.4637 - val_loss: 2.4204
Epoch 3/10
549/549 [==============================] - 45s 81ms/step - loss: 2.2899 - val_loss: 2.3212
Epoch 4/10
549/549 [==============================] - 42s 77ms/step - loss: 2.1348 - val_loss: 2.2520
Epoch 5/10
549/549 [==============================] - 40s 72ms/step - loss: 1.9836 - val_loss: 2.1935
Epoch 6/10
549/549 [==============================] - 40s 72ms/step - loss: 1.8329 - val_loss: 2.1498TA: 0s 
Epoch 7/10
549/549 [==============================] - 40s 72ms/step - loss: 1.6891 - val_loss: 2.1138
Epoch 8/10
549/549 [==============================] - 40s 72ms/step - loss: 1.5554 - val_loss: 2.0941
Epoch 9/10
549/549 [==============================] - 40s 72ms/step - loss: 1.4386 - val_loss: 2.0850
Epoch 10/10
549/549 [==============================] - 40s 72ms/step - loss

In [19]:
def generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [26]:
generate_text(model, tokenizer, init_sentence="<start> Sun")

'<start> sun is <unk> , moon is gone <end> '

1. Word=1600, batch 2, embedding size=256 ,lr=0.001 => overfitting    
2. Word=1600, batch 2, embedding size=256, lr=0.05 => overfitting    
<br/>   
3. Word=1500, batch 2, embedding size=256, lr=0.001 => overfitting(Epoch 7_2.23~)
4. Word=1500, batch 2, embedding size=256, lr=0.05 => overfitting(Epoch 5 3.128~)    
<br/>   
5. Word=1400, batch 2, embedding size=256, lr=0.001 => overfitting(Epoch 7 2.21~)    
<br/>    
6. Word=1200, batch 2, embedding size=256, lr=0.001 => overfitting(Epoch 7 2.12~)
7. Word=1200, batch 1 (LSTM ~ Dense), embedding size=256, lr=0.001(Epoch 8 2.20~)   
<br/>   
```최종```
8. Word=1200, batch 1 (LSTM1 ~ LSTM2), embedding size=256, lr=0.001    
-> **val_loss: 2.0889**



### 정리
- Batch Normalization를 활용하여 성능을 더 좋게 만들 수 있었음
    - BN을 layer에 두 번 활용할 경우 Overfitting이 더 자주 일어남
    - BN을 한 번은 LSTM_2과 Dense 레이어 사이에 넣어서 실행
    - 나머지 한 번은 LSTM_1과 LSTM_2 레이어 사이에 넣어서 실행
- 전체 단어의 개수는 1201개 이상 진행할 시 쉽게 Overfitting이 일어남
    - Word 수를 줄일 수록 더 좋은 모형을 드러냄
     
 <br/>   
 ```결론```
 - word = 1200, batch 1 (LSTM_1 ~ LSTM_2 layer), embedding size = 256, learning rate = 0.001
 - **val_loss: 2.0889**
 - 요구하는 val_loss 2.2 이하 맞춤